# Machine Learning Pipeline Exploration Jupyter Notebook

This jupyter notebook is used to explore how different ML models perform and to finetune hyperparameters.

## Import Libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/jonas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jonas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jonas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Step 1: Load the data

In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table("messages", engine)

categories = list(df.columns)
categories.remove("id")
categories.remove("message")
categories.remove("original")
categories.remove("genre")

X = df["message"]
Y = df[categories]

## Step 2: Write

In [3]:


def tokenize(text):
    # Normalize text by converting to lowercase and removing punctuation
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stop words
    tokens = [token for token in tokens if token not in stopwords.words('english')]
    
    # Lemmatize tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:


# Build the machine learning pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Train the pipeline
pipeline.fit(X_train, Y_train)

/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f5d7736c1f0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
# Predict on the testing set
Y_pred = pipeline.predict(X_test)

# Print classification report for each category
for i, category in enumerate(categories):
    print(f"Category: {category}\n")
    print(classification_report(Y_test.iloc[:, i], Y_pred[:, i]))

Category: floods

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      4833
           1       0.89      0.46      0.61       411

    accuracy                           0.95      5244
   macro avg       0.92      0.73      0.79      5244
weighted avg       0.95      0.95      0.95      5244

Category: hospitals

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      5198
           1       0.00      0.00      0.00        46

    accuracy                           0.99      5244
   macro avg       0.50      0.50      0.50      5244
weighted avg       0.98      0.99      0.99      5244

Category: aid_related

              precision    recall  f1-score   support

           0       0.79      0.85      0.82      3113
           1       0.75      0.68      0.71      2131

    accuracy                           0.78      5244
   macro avg       0.77      0.76      0.77      5244
weighted avg

/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defi

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      4766
           1       0.88      0.76      0.82       478

    accuracy                           0.97      5244
   macro avg       0.93      0.88      0.90      5244
weighted avg       0.97      0.97      0.97      5244

Category: death

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5007
           1       0.88      0.19      0.31       237

    accuracy                           0.96      5244
   macro avg       0.92      0.59      0.64      5244
weighted avg       0.96      0.96      0.95      5244

Category: buildings

              precision    recall  f1-score   support

           0       0.95      1.00      0.98      4977
           1       0.90      0.10      0.19       267

    accuracy                           0.95      5244
   macro avg       0.93      0.55      0.58      5244
weighted avg       0.95      0.95   

/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defi

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
from multiprocessing import cpu_count
from joblib import parallel_backend
from sklearn.model_selection import GridSearchCV

cpus = cpu_count() - 2

# Define the parameter grid
parameters = {
    'vect__max_df': (0.8, 0.9, 1.0),
    'clf__estimator__n_estimators': [50, 100, 200],
}
with parallel_backend('multiprocessing'):
    # Initialize GridSearchCV
    cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=cpus, verbose=2)

    # Fit and tune model
    cv.fit(X_train, Y_train)

/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Fitting 5 folds for each of 9 candidates, totalling 45 fits


/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_patter

[CV] END ..clf__estimator__n_estimators=50, vect__max_df=0.9; total time= 6.4min


/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...

/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END .clf__estimator__n_estimators=200, vect__max_df=0.9; total time=28.6min


/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[CV] END .clf__estimator__n_estimators=200, vect__max_df=0.9; total time=25.2min
[CV] END .clf__estimator__n_estimators=200, vect__max_df=1.0; total time=25.5min
[CV] END .clf__estimator__n_estimators=200, vect__max_df=1.0; total time=26.2min
[CV] END .clf__estimator__n_estimators=200, vect__max_df=1.0; total time=20.2min
[CV] END .clf__estimator__n_estimators=200, vect__max_df=1.0; total time=19.6min
[CV] END .clf__estimator__n_estimators=200, vect__max_df=1.0; total time=18.7min


/home/jonas/Dokumente/arbeit/ecodia-gmbh/gitrepos/disaster_response_pipeline_project/venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
cv.best_params_

{'clf__estimator__n_estimators': 100, 'vect__max_df': 1.0}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.